In [12]:
#从数据库获取数据 更新toh_get_ toh_use相关原始数据
import sys
import os
import codecs
from sys import argv
import pymysql
import pandas as pd

config = dict(host='125.124.0.156', user='chencheng', password='Cc$toh$db1', port= 3306,database = "toh", cursorclass=pymysql.cursors.DictCursor, charset='UTF8')

conn = pymysql.Connect(**config)
conn.autocommit(1)
cursor = conn.cursor()

dic = {};

def run_out():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_use_316`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_use.csv", index=False);
    out_data['system'] = out_data['channel'];
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_use2.csv", index=False);


def run_get():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_get_316_copy`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_get.csv", index=False);
    out_data['system'] = out_data['channel'];
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_get2.csv", index=False);

if __name__ == '__main__':
    '''
    main
    '''
    run_get();
    run_out();

          desc system             channel
0           金币    周活动           顺序兑换活动活动赠
1         时光之沙    周活动           顺序兑换活动活动赠
2           卡牌    周活动           顺序兑换活动活动赠
3     固定英雄卡牌碎片    周活动           顺序兑换活动活动赠
4           装备    周活动           顺序兑换活动活动赠
..         ...    ...                 ...
535    骰子 dice    周活动  登陆有奖(活动期间登录拿东西)活动赠
536        虚空石    周活动  登陆有奖(活动期间登录拿东西)活动赠
537  lemon_tea    周活动  登陆有奖(活动期间登录拿东西)活动赠
538  stopwatch    周活动  登陆有奖(活动期间登录拿东西)活动赠
539   egg_tart    周活动  登陆有奖(活动期间登录拿东西)活动赠

[540 rows x 3 columns]
                      desc system      channel
0                       钻石    周活动    顺序兑换活动活动赠
1                      灵魂石    周活动    顺序兑换活动活动赠
2                lemon_tea    周活动    顺序兑换活动活动赠
3                stopwatch    周活动    顺序兑换活动活动赠
4    horse searach stamina     龙岛         天马探索
..                     ...    ...          ...
142                    许愿币     转盘         使用转盘
143                     钻石     工会         创建公会
144                 进背包的宝箱    周活动          开宝箱
14

In [7]:
#从toh_get_ toh_use相关原始数据 整理出作图需要的out.csv数据
import networkx as nx
import pandas as pd

l_source = set({});
l_funct = set({});
l_system = set({});

df_s_f = pd.read_csv('./func_system.csv');
df_s_f.rename(columns={'system': 'source', 'function' : 'target'}, inplace=True);
Graphtype = nx.DiGraph()
G0 = nx.from_pandas_edgelist(df_s_f, create_using=Graphtype)
for node in list(G0):
    nbrs = nx.neighbors(G0, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
        l_funct.add(node);
    else:
       #是资源类型
        l_system.add(node);


df = pd.read_csv('./toh_use2.csv')
df.rename(columns={'desc': 'source', 'system' : 'target'}, inplace=True);
Graphtype = nx.DiGraph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)
list(G)

list(G.edges())
for node in list(G):
    nbrs = nx.neighbors(G, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
        l_funct.add(node);
    else:
       #是资源类型
        l_source.add(node);

df2 = pd.read_csv('./toh_get2.csv')
df2.rename(columns={'desc': 'target', 'system' : 'source'}, inplace=True);
Graphtype = nx.DiGraph()
G2 = nx.from_pandas_edgelist(df2, create_using=Graphtype)
list(G2.edges())
for node in list(G2):
    nbrs = nx.neighbors(G2, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
       #是资源类型
       l_source.add(node);
    else:
        l_funct.add(node);
       #是功能类型

# 只取某系统/某资源对应系统
U_all=nx.DiGraph()
U_all.add_edges_from(G.edges())
U_all.add_edges_from(G2.edges());
U_all.add_nodes_from(G.nodes())
U_all.add_nodes_from(G2.nodes())
U_all.add_edges_from(G0.edges())
U_all.add_nodes_from(G0.nodes())

# U_all.remove_node('周活动');
# U_all.remove_node('月活动');
# U_all.remove_node('月礼包');
# U_all.remove_node('礼包');
# l_funct.remove('周活动');
# l_funct.remove('月活动');
# l_funct.remove('月礼包');
# l_funct.remove('礼包');


# print("all system:", list(l_funct));

def out_csv(_U_all, out_path = "to_io_draw.csv"):
    df_out = pd.DataFrame([], columns=['id','name','shape','fill','refs']);
    i = 0;
    for node in list(_U_all):
        nbrs = nx.neighbors(_U_all, node)
        shape_ = "ellipse" if(node in l_source) else ("rectangle" if (node in l_funct) else "rhombus")
        fill_ = "#b85450" if(node in l_source) else ("#82b366" if(node in l_funct) else "#dae8fc")
        row = [node, node, shape_, fill_ ,','.join(list(set(nbrs)))]
        df_out.loc[i]=row
        i+=1;
    df_out.to_csv(out_path, index=False);

out_csv(U_all, "./draw_io_out/all.csv");
for s in l_source:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/source/to_io_draw_" + s + ".csv");

for s in l_funct:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/func/to_io_draw_func_" + s + ".csv");

for s in l_system:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    sss = set({});
    for n in nx.all_neighbors(U_all, s):
        subnodes = subnodes + list(nx.all_neighbors(U_all, n));
    
    sub = U_all.subgraph(subnodes)
#     sub = nx.ego_graph(U_all, s, radius=2)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/system/to_io_draw_sys_" + s + ".csv");


['节日礼包', '公会礼包打开', '特惠礼包', '挂机', '登陆有奖(活动期间登录拿东西)活动赠', '顺序兑换活动活动赠', 'stopwatch']
edgess stopwatch ['登陆有奖(活动期间登录拿东西)活动赠', 'stopwatch', '公会礼包打开', '节日礼包', '挂机', '特惠礼包', '顺序兑换活动活动赠'] [('登陆有奖(活动期间登录拿东西)活动赠', 'stopwatch'), ('stopwatch', '顺序兑换活动活动赠'), ('公会礼包打开', 'stopwatch'), ('节日礼包', 'stopwatch'), ('挂机', 'stopwatch'), ('特惠礼包', 'stopwatch')]
['巨龙之岛金苹果初始化', '巨龙之岛建筑产出', '巨龙之岛买苹果', '巨龙之岛好友助战', '巨龙之岛打boss消耗', '极乐之境金苹果']
edgess 极乐之境金苹果 ['巨龙之岛好友助战', '极乐之境金苹果', '巨龙之岛打boss消耗', '巨龙之岛买苹果', '巨龙之岛金苹果初始化', '巨龙之岛建筑产出'] [('极乐之境金苹果', '巨龙之岛好友助战'), ('极乐之境金苹果', '巨龙之岛打boss消耗'), ('巨龙之岛买苹果', '极乐之境金苹果'), ('巨龙之岛金苹果初始化', '极乐之境金苹果'), ('巨龙之岛建筑产出', '极乐之境金苹果')]
['符文升级', '水晶抽次数活动赠', '高星置换-7星', '高星置换-8星', '高级置换材料 荷鲁斯之眼']
edgess 高级置换材料 荷鲁斯之眼 ['高星置换-8星', '高星置换-7星', '高级置换材料 荷鲁斯之眼', '水晶抽次数活动赠', '符文升级'] [('高级置换材料 荷鲁斯之眼', '高星置换-7星'), ('高级置换材料 荷鲁斯之眼', '高星置换-8星'), ('水晶抽次数活动赠', '高级置换材料 荷鲁斯之眼'), ('符文升级', '高级置换材料 荷鲁斯之眼')]
['节日礼包', '阵营先知进度活动活动赠', '旅行大富翁活动赠', '开宝箱', '进背包的宝箱']
edgess 进背包的宝箱 ['阵营先知进度活动活动赠', '开宝箱', '进背包的宝箱', '节日礼包', '旅行大富

['新的黑市商店购买', '先知召唤-恶魔', '先知召唤-天灾', '先知召唤-人类', '训练营领奖', '成就领奖', '先知召唤-光暗', '竞技场商店购买', '酒馆获得', '先知召唤-暗夜', '先知置换-人类', '先知置换-天灾', '先知置换-暗夜', '先知置换-恶魔', '先知祝福，英雄置换']
edgess 先知祝福，英雄置换 ['先知置换-暗夜', '酒馆获得', '先知置换-人类', '竞技场商店购买', '先知召唤-恶魔', '训练营领奖', '先知召唤-人类', '先知召唤-暗夜', '新的黑市商店购买', '先知置换-天灾', '成就领奖', '先知置换-恶魔', '先知祝福，英雄置换', '先知召唤-天灾', '先知召唤-光暗'] [('酒馆获得', '先知祝福，英雄置换'), ('竞技场商店购买', '先知祝福，英雄置换'), ('先知召唤-恶魔', '先知祝福，英雄置换'), ('训练营领奖', '先知祝福，英雄置换'), ('先知召唤-人类', '先知祝福，英雄置换'), ('先知召唤-暗夜', '先知祝福，英雄置换'), ('新的黑市商店购买', '先知祝福，英雄置换'), ('成就领奖', '先知祝福，英雄置换'), ('先知祝福，英雄置换', '先知置换-人类'), ('先知祝福，英雄置换', '先知置换-天灾'), ('先知祝福，英雄置换', '先知置换-暗夜'), ('先知祝福，英雄置换', '先知置换-恶魔'), ('先知召唤-天灾', '先知祝福，英雄置换'), ('先知召唤-光暗', '先知祝福，英雄置换')]
['买爬塔体力', '爬塔战斗', '环境之塔的体力']
edgess 环境之塔的体力 ['爬塔战斗', '买爬塔体力', '环境之塔的体力'] [('买爬塔体力', '环境之塔的体力'), ('环境之塔的体力', '爬塔战斗')]
['挂机', '天马完成任务', '酒馆增加任务', '酒馆卷轴']
edgess 酒馆卷轴 ['酒馆卷轴', '挂机', '酒馆增加任务', '天马完成任务'] [('酒馆卷轴', '酒馆增加任务'), ('挂机', '酒馆卷轴'), ('天马完成任务', '酒馆卷轴')]
['顺序兑换活动活动赠', '等级限时礼包', '地牢关卡掉落', '新的黑市商店购买', '

['好友送心', '成就领奖', '爱心抽卡', '爱心']
edgess 爱心 ['爱心', '好友送心', '成就领奖', '爱心抽卡'] [('爱心', '爱心抽卡'), ('好友送心', '爱心'), ('成就领奖', '爱心')]
['顺序兑换活动活动赠', '新的黑市商店购买', '先知召唤-恶魔', '英雄分解商店购买', '先知召唤-天灾', '瓦尔哈拉之邀活动赠', '获得捐出的卡牌碎片', '先知召唤-人类', '藏宝图2活动赠', '训练营领奖', '神圣商店购买', '转盘次数活动赠', '转盘商店购买', '英雄委托活动赠', 'vip礼包', '使用高级转盘', '水晶抽次数活动赠', '爬塔战斗', '成就领奖', '高级转盘活动活动赠', '在线奖励', '先知召唤-光暗', '旅行大富翁活动赠', '先知召唤活动活动赠', '先知召唤-暗夜', '签到', '工会商店购买', '远征商店购买', '使用转盘', '开宝箱', 'pve过关', '首冲送礼包', '卡牌碎片合成', '退款后扣除', '卖道具', '捐出卡牌碎片', '固定英雄卡牌碎片']
edgess 固定英雄卡牌碎片 ['神圣商店购买', '使用转盘', '退款后扣除', '先知召唤-天灾', '英雄分解商店购买', '高级转盘活动活动赠', 'pve过关', 'vip礼包', '先知召唤-人类', '捐出卡牌碎片', '获得捐出的卡牌碎片', '新的黑市商店购买', '转盘商店购买', '使用高级转盘', '水晶抽次数活动赠', '英雄委托活动赠', '旅行大富翁活动赠', '卖道具', '顺序兑换活动活动赠', '瓦尔哈拉之邀活动赠', '转盘次数活动赠', '首冲送礼包', '远征商店购买', '成就领奖', '爬塔战斗', '签到', '先知召唤-恶魔', '训练营领奖', '在线奖励', '先知召唤-暗夜', '开宝箱', '固定英雄卡牌碎片', '工会商店购买', '卡牌碎片合成', '先知召唤活动活动赠', '藏宝图2活动赠', '先知召唤-光暗'] [('神圣商店购买', '固定英雄卡牌碎片'), ('使用转盘', '固定英雄卡牌碎片'), ('先知召唤-天灾', '固定英雄卡牌碎片'), ('英雄分解商店购买', '固定英雄卡牌碎片'), ('高

['周活动', '金币', '时光之沙', '卡牌升级石头', '神器碎片', '恶魔之魂， 升级符文用', '混沌石， 符文商店用', '虚空击杀奖励']
edgess 虚空击杀奖励 ['卡牌升级石头', '神器碎片', '虚空击杀奖励', '周活动', '混沌石， 符文商店用', '恶魔之魂， 升级符文用', '时光之沙', '金币'] [('虚空击杀奖励', '金币'), ('虚空击杀奖励', '时光之沙'), ('虚空击杀奖励', '卡牌升级石头'), ('虚空击杀奖励', '神器碎片'), ('虚空击杀奖励', '恶魔之魂， 升级符文用'), ('虚空击杀奖励', '混沌石， 符文商店用'), ('周活动', '虚空击杀奖励')]
['先知宝珠，先知门', '先知之树', '固定英雄卡牌碎片', '随机英雄碎片', '先知祝福，英雄置换', '先知召唤-天灾']
edgess 先知召唤-天灾 ['先知之树', '固定英雄卡牌碎片', '随机英雄碎片', '先知宝珠，先知门', '先知祝福，英雄置换', '先知召唤-天灾'] [('先知之树', '先知召唤-天灾'), ('先知宝珠，先知门', '先知召唤-天灾'), ('先知召唤-天灾', '固定英雄卡牌碎片'), ('先知召唤-天灾', '随机英雄碎片'), ('先知召唤-天灾', '先知祝福，英雄置换')]
['钻石', '其他养成', '金币', '点金手']
edgess 点金手 ['金币', '钻石', '点金手', '其他养成'] [('钻石', '点金手'), ('点金手', '金币'), ('其他养成', '点金手')]
['活跃活动', '钻石', '抽卡卷轴', 'REDEMPTION_CODE']
edgess REDEMPTION_CODE ['抽卡卷轴', '钻石', 'REDEMPTION_CODE', '活跃活动'] [('REDEMPTION_CODE', '钻石'), ('REDEMPTION_CODE', '抽卡卷轴'), ('活跃活动', 'REDEMPTION_CODE')]
['礼包', '钻石', '金币', '时光之沙', '固定英雄卡牌碎片', '装备', '卡牌升级石头', '卡牌进阶石头', '随机英雄碎片', '先知宝珠，先知门', '抽卡卷轴', '公

['周活动', '钻石', '先知宝珠，先知门', '神器碎片', '神圣纹章(辉煌商店货币)', '装备附魔书', '虚空排行奖励']
edgess 虚空排行奖励 ['神器碎片', '神圣纹章(辉煌商店货币)', '周活动', '先知宝珠，先知门', '钻石', '虚空排行奖励', '装备附魔书'] [('周活动', '虚空排行奖励'), ('虚空排行奖励', '钻石'), ('虚空排行奖励', '先知宝珠，先知门'), ('虚空排行奖励', '神器碎片'), ('虚空排行奖励', '神圣纹章(辉煌商店货币)'), ('虚空排行奖励', '装备附魔书')]
['钻石', '金币', '地牢', '卡牌升级石头', '卡牌进阶石头', '随机英雄碎片', '许愿币', '竞技场门票', '先知宝珠，先知门', '抽卡卷轴', '地牢买商人']
edgess 地牢买商人 ['卡牌升级石头', '卡牌进阶石头', '地牢', '竞技场门票', '随机英雄碎片', '先知宝珠，先知门', '许愿币', '抽卡卷轴', '钻石', '金币', '地牢买商人'] [('地牢', '地牢买商人'), ('钻石', '地牢买商人'), ('金币', '地牢买商人'), ('地牢买商人', '卡牌升级石头'), ('地牢买商人', '卡牌进阶石头'), ('地牢买商人', '随机英雄碎片'), ('地牢买商人', '许愿币'), ('地牢买商人', '竞技场门票'), ('地牢买商人', '先知宝珠，先知门'), ('地牢买商人', '抽卡卷轴')]
['灵魂石', '分解功能', '固定英雄卡牌碎片', '英雄分解商店购买']
edgess 英雄分解商店购买 ['英雄分解商店购买', '灵魂石', '分解功能', '固定英雄卡牌碎片'] [('英雄分解商店购买', '固定英雄卡牌碎片'), ('灵魂石', '英雄分解商店购买'), ('分解功能', '英雄分解商店购买')]
['周活动', '固定英雄卡牌碎片', '装备', '随机英雄碎片', '抽卡卷轴', '神圣纹章(辉煌商店货币)', '龙纹币', '高级转盘活动活动赠']
edgess 高级转盘活动活动赠 ['龙纹币', '高级转盘活动活动赠', '固定英雄卡牌碎片', '神圣纹章(辉煌商店货币)', '周活动', '随

['金币', '装备', '装备附魔书', '装备系统', '装备', '装备附魔']
edgess 装备附魔 ['装备附魔', '装备', '装备系统', '金币', '装备附魔书'] [('装备附魔', '装备'), ('装备', '装备附魔'), ('装备系统', '装备附魔'), ('金币', '装备附魔'), ('装备附魔书', '装备附魔')]
['金币', '装备', '装备系统', '装备', '装备合成']
edgess 装备合成 ['装备系统', '金币', '装备合成', '装备'] [('装备系统', '装备合成'), ('金币', '装备合成'), ('装备合成', '装备'), ('装备', '装备合成')]
['虚空石', '月活动', '金币', '时光之沙', '卡牌升级石头', '恶魔之魂， 升级符文用', '混沌石， 符文商店用', '虚空普通战斗奖励']
edgess 虚空普通战斗奖励 ['卡牌升级石头', '虚空普通战斗奖励', '混沌石， 符文商店用', '恶魔之魂， 升级符文用', '虚空石', '时光之沙', '金币', '月活动'] [('虚空普通战斗奖励', '金币'), ('虚空普通战斗奖励', '时光之沙'), ('虚空普通战斗奖励', '卡牌升级石头'), ('虚空普通战斗奖励', '恶魔之魂， 升级符文用'), ('虚空普通战斗奖励', '混沌石， 符文商店用'), ('虚空石', '虚空普通战斗奖励'), ('月活动', '虚空普通战斗奖励')]
['工会', '固定英雄卡牌碎片', '随机英雄碎片', '获得捐出的卡牌碎片']
edgess 获得捐出的卡牌碎片 ['工会', '随机英雄碎片', '获得捐出的卡牌碎片', '固定英雄卡牌碎片'] [('工会', '获得捐出的卡牌碎片'), ('获得捐出的卡牌碎片', '固定英雄卡牌碎片'), ('获得捐出的卡牌碎片', '随机英雄碎片')]
['工会', '钻石', '金币', '卡牌升级石头', '随机英雄碎片', '公会币', 'stopwatch', '公会礼包打开']
edgess 公会礼包打开 ['卡牌升级石头', 'stopwatch', '公会币', '工会', '随机英雄碎片', '公会礼包打开', '钻石', '金币'] [('工会', 

['好友', '钻石', '抽卡卷轴', 'FRIEND_MAZE_BOX']
edgess FRIEND_MAZE_BOX ['好友', '抽卡卷轴', '钻石', 'FRIEND_MAZE_BOX'] [('好友', 'FRIEND_MAZE_BOX'), ('FRIEND_MAZE_BOX', '钻石'), ('FRIEND_MAZE_BOX', '抽卡卷轴')]
['神圣纹章(辉煌商店货币)', '神圣商店', '固定英雄卡牌碎片', '神圣商店购买']
edgess 神圣商店购买 ['神圣商店购买', '固定英雄卡牌碎片', '神圣商店', '神圣纹章(辉煌商店货币)'] [('神圣商店购买', '固定英雄卡牌碎片'), ('神圣商店', '神圣商店购买'), ('神圣纹章(辉煌商店货币)', '神圣商店购买')]
['金币', '卡牌升级石头', '英雄系统', '卡牌升级']
edgess 卡牌升级 ['金币', '卡牌升级', '卡牌升级石头', '英雄系统'] [('金币', '卡牌升级'), ('卡牌升级石头', '卡牌升级'), ('英雄系统', '卡牌升级')]
['许愿币', '转盘', '金币', '时光之沙', '固定英雄卡牌碎片', '装备', '卡牌升级石头', '卡牌进阶石头', '随机英雄碎片', '转盘商场幸运币', '使用转盘']
edgess 使用转盘 ['卡牌升级石头', '卡牌进阶石头', '固定英雄卡牌碎片', '随机英雄碎片', '转盘商场幸运币', '装备', '使用转盘', '许愿币', '时光之沙', '转盘', '金币'] [('使用转盘', '金币'), ('使用转盘', '时光之沙'), ('使用转盘', '固定英雄卡牌碎片'), ('使用转盘', '装备'), ('使用转盘', '卡牌升级石头'), ('使用转盘', '卡牌进阶石头'), ('使用转盘', '随机英雄碎片'), ('使用转盘', '转盘商场幸运币'), ('许愿币', '使用转盘'), ('转盘', '使用转盘')]
['卡牌', '分解功能', '金币', '时光之沙', '卡牌升级石头', '卡牌进阶石头', '灵魂石', '卡牌分解']
edgess 卡牌分解 ['卡牌升级石头', '卡牌进阶石头', '卡牌', '分解功能',

['先知宝珠，先知门', '先知之树', '固定英雄卡牌碎片', '随机英雄碎片', '先知祝福，英雄置换', '先知召唤-光暗']
edgess 先知召唤-光暗 ['先知之树', '固定英雄卡牌碎片', '随机英雄碎片', '先知宝珠，先知门', '先知祝福，英雄置换', '先知召唤-光暗'] [('先知之树', '先知召唤-光暗'), ('先知宝珠，先知门', '先知召唤-光暗'), ('先知召唤-光暗', '固定英雄卡牌碎片'), ('先知召唤-光暗', '随机英雄碎片'), ('先知召唤-光暗', '先知祝福，英雄置换')]
edgess 周活动 ['stopwatch', '阵营先知进度活动活动赠', '虚空击杀奖励', '高级置换材料 荷鲁斯之眼', '神圣纹章(辉煌商店货币)', '进背包的宝箱', '周活动', '装备', '虚空石', '虚空排行奖励', '卡牌进阶石头', '指定皮肤碎片(指定的skin id)', '符文升级', '高级转盘活动活动赠', '卡牌合成某星卡牌(有领取次数限制)活动赠', '神器碎片', '回收旅行骰子', '混沌石， 符文商店用', '随机英雄碎片', '水晶抽次数活动赠', 'lemon_tea', '许愿币', '英雄委托活动赠', '旅行大富翁活动赠', '高级许愿币', '买活动副本', '卖道具', '顺序兑换活动活动赠', '虚空购买石头', '登陆有奖(活动期间登录拿东西)活动赠', '卡牌升级石头', 'RUNE', '公会币', '龙纹币', '瓦尔哈拉之邀活动赠', '转盘次数活动赠', '先知宝珠，先知门', '抽卡卷轴', '灵魂石', '金币', '抽到特定英雄活动赠', '开宝箱', '卡牌', '固定英雄卡牌碎片', '骰子 dice', '竞技场门票', '恶魔之魂， 升级符文用', '随机皮肤碎片', '登陆有奖(活动期间，注册时间在7天以内的玩家登录拿东西)活动赠', '钻石', '先知召唤活动活动赠', '藏宝图2活动赠', '时光之沙', '装备附魔书', 'egg_tart', 'paddle2'] [('stopwatch', '顺序兑换活动活动赠'), ('阵营先知进度活动活动赠', '进背包的宝箱'), ('虚空击杀奖励', '金币'), ('虚空击杀奖励', '时

edgess 神圣商店 ['固定英雄卡牌碎片', '神圣商店购买', '神圣商店', '神圣纹章(辉煌商店货币)'] [('神圣商店购买', '固定英雄卡牌碎片'), ('神圣商店', '神圣商店购买'), ('神圣纹章(辉煌商店货币)', '神圣商店购买')]
edgess 龙岛 ['极乐之境金苹果', '巨龙之岛买苹果', '极乐之境金羽毛', '巨龙之岛建筑产出', '卡牌进阶石头', '龙岛', '天马升级任务', '神器碎片', '天马探索', '混沌石， 符文商店用', '巨龙之岛升级建筑', '巨龙之岛金苹果初始化', '巨龙之岛击杀boss获得', 'horse searach stamina', '随机英雄碎片', '高级许愿币', '天马买耐久', '酒馆卷轴', '巨龙之岛刷新boss', '卡牌升级石头', '龙纹币', '巨龙之岛好友助战', '巨龙之岛打boss消耗', '先知宝珠，先知门', '天马完成任务', '抽卡卷轴', 'HORSE_EXP', '金币', 'HORSE_START', '卡牌', '恶魔之魂， 升级符文用', '随机皮肤碎片', '钻石', '巨龙之岛重置建筑', '时光之沙', '天马奇遇'] [('极乐之境金苹果', '巨龙之岛好友助战'), ('极乐之境金苹果', '巨龙之岛打boss消耗'), ('巨龙之岛买苹果', '极乐之境金苹果'), ('极乐之境金羽毛', '巨龙之岛升级建筑'), ('巨龙之岛建筑产出', '钻石'), ('巨龙之岛建筑产出', '金币'), ('巨龙之岛建筑产出', '极乐之境金苹果'), ('巨龙之岛建筑产出', '龙纹币'), ('龙岛', '天马探索'), ('龙岛', '巨龙之岛金苹果初始化'), ('龙岛', '巨龙之岛建筑产出'), ('龙岛', '巨龙之岛刷新boss'), ('龙岛', '巨龙之岛重置建筑'), ('龙岛', '巨龙之岛买苹果'), ('龙岛', '天马买耐久'), ('龙岛', '天马升级任务'), ('龙岛', '巨龙之岛升级建筑'), ('龙岛', '天马奇遇'), ('龙岛', 'HORSE_START'), ('龙岛', '巨龙之岛击杀boss获得'), ('龙岛', '巨龙之岛好友助战'), ('龙岛', '巨龙之岛打boss消耗'

edgess 抽卡 ['爱心', '卡牌', '抽卡', '爱心抽卡', '抽卡卷轴', '高级抽卡', '低级抽卡', '能量抽卡', '钻石'] [('爱心', '爱心抽卡'), ('抽卡', '爱心抽卡'), ('抽卡', '低级抽卡'), ('抽卡', '能量抽卡'), ('抽卡', '高级抽卡'), ('爱心抽卡', '卡牌'), ('抽卡卷轴', '低级抽卡'), ('抽卡卷轴', '高级抽卡'), ('高级抽卡', '卡牌'), ('低级抽卡', '卡牌'), ('能量抽卡', '卡牌'), ('钻石', '高级抽卡')]
edgess 工会 ['stopwatch', '公会捐献', '装备', '公会boss击杀奖励', '工会战普通排名', '卡牌进阶石头', '工会商店刷新', '神器碎片', '公会改名', '捐出卡牌碎片', '获得捐出的卡牌碎片', '公会礼包打开', '随机英雄碎片', '卡牌升级石头', '公会币', '购买公会boss复活', '公会挖矿', '升级许愿树', '创建公会', '升级公会订单', '金币', '重置公会科技', '捐出卡牌碎片获得奖励', '解锁升级公会科技', '固定英雄卡牌碎片', '公会boss排名奖励', '公会签到', '工会', '工会商店购买', '时光之沙', '钻石', '公会boss战斗'] [('公会捐献', '公会币'), ('公会boss击杀奖励', '金币'), ('公会boss击杀奖励', '公会币'), ('工会战普通排名', '金币'), ('工会战普通排名', '公会币'), ('获得捐出的卡牌碎片', '固定英雄卡牌碎片'), ('获得捐出的卡牌碎片', '随机英雄碎片'), ('公会礼包打开', '钻石'), ('公会礼包打开', '金币'), ('公会礼包打开', '卡牌升级石头'), ('公会礼包打开', '随机英雄碎片'), ('公会礼包打开', '公会币'), ('公会礼包打开', 'stopwatch'), ('随机英雄碎片', '捐出卡牌碎片'), ('公会币', '解锁升级公会科技'), ('公会币', '工会商店刷新'), ('公会币', '工会商店购买'), ('公会挖矿', '金币'), ('公会挖矿', '时光之沙'), ('公会挖矿', 

In [ ]:
test_dup = pd.read_csv("toh_get2.csv");
print(test_dup);
g = test_dup.groupby(['desc','system']).groups
print(len(g))
# test_dup.drop_duplicates();
# print(test_dup)

In [ ]:
df_get = pd.read_csv('./toh_get.csv');
# print(df_get.describe)
set_get = set(df_get['source'])
# print(len(set_get));

df_use = pd.read_csv('./toh_use.csv');
# print(df_use.describe)
set_use = set(df_use['source'])
# print(len(set_use));
func_ = pd.read_csv('./func_system.csv');
sys = func_["system"];
ss_sys = set(sys);
print(len(ss_sys));
list(ss_sys)
ss_all = set(list(set_get) + list(set_use));
print(ss_all)
print(len(ss_all))